This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
 

In [1]:
figname = '../paper/fig_params'

In [2]:
%run 0_parameters.ipynb


This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      
Using filename= ../data/2019-03-15
{'w': 28, 'minibatch_size': 100, 'train_batch_size': 600, 'test_batch_size': 1000, 'noise_batch_size': 1000, 'mean': 0.1307, 'std': 0.3081, 'N_pic': 128, 'offset_std': 30, 'offset_max': 35, 'noise': 0.7, 'contrast': 0.8, 'sf_0': 0.2, 'B_sf': 0.3, 'N_theta': 6, 'N_azimuth': 16, 'N_eccentricity': 10, 'N_phase': 2, 'rho': 1.41, 'bias_deconv': True, 'p_dropout': 0.5, 'dim1': 1000, 'dim2': 1000, 'lr': 0.0001, 'do_adam': True, 'bn1_bn_momentum': 0.0, 'bn2_bn_momentum': 0.0, 'momentum': 0.1, 'epochs': 10, 'num_processes': 1, 'no_cuda': True, 'log_interval': 100, 'verbose': 1, 'filename': '../data/2019-03-15', 'seed': 2019, 'N_cv': 10}
Overwriting train.py
2019-03-15T17:19:15+01:00

CPython 3.7.2
IPython 7.3.0

numpy 1.16.2
matplotlib 3.0.3
torch 1.0.1.post2

compiler   : Clang 10.0.0 (clang-1000.11.45.5)
s

In [3]:
%%writefile params.py
import torch
from main import init, MetaML

opts = dict(filename='../data/2019-03-16', verbose=0, log_interval=0)
#opts = dict(filename='debug', verbose=0, log_interval=0)

print(50*'-')
print(' parameter scan')
print(50*'-')

args = init(**opts)
from where import Where as ML
from what import WhatNet
ml = ML(args)
if False:
    print(50*'-')
    print('Default parameters')
    print(50*'-')

    ml.train(path=args.filename)
    # ml.main(path=args.filename)

if True:
    args = init(**opts)
    mml = MetaML(args)
    if torch.cuda.is_available():
        mml.scan('no_cuda', [True, False])

    args = init(**opts)
    mml = MetaML(args)
    mml.scan('bias_deconv', [True, False])
   

for base in [2, 8] if not args.filename == '../data/debug' else [2]:
    print(50*'-')
    print(' base=', base)
    print(50*'-')

    print(50*'-')
    print(' parameter scan : data')
    print(50*'-')
    args = init(**opts)
    mml = MetaML(args, base=base)
    for parameter in ['sf_0', 'B_sf', 'offset_std', 'noise', 'contrast']:
        mml.parameter_scan(parameter)
        
    # TODO:  'N_theta': 6, 'N_azimuth': 16, 'N_eccentricity': 10, 'rho': 1.41,

    print(50*'-')
    print(' parameter scan : network')
    print(50*'-')
    args = init(**opts)
    mml = MetaML(args)
    for parameter in ['dim1',
                      'bn1_bn_momentum',
                      'dim2',
                      'bn2_bn_momentum',
                      'p_dropout']:
        mml.parameter_scan(parameter)

    print(' parameter scan : learning ')
    args = init(**opts)
    mml = MetaML(args, base=base)
    print(50*'-')
    print('Using SGD')
    print(50*'-')
    for parameter in ['lr', 'momentum', 'minibatch_size', 'epochs']:
        mml.parameter_scan(parameter)
    print(50*'-')
    print('Using ADAM')
    print(50*'-')
    args = init(**opts)
    args.do_adam = True
    mml = MetaML(args, tag='adam')
    for parameter in ['lr', 'momentum', 'minibatch_size', 'epochs']:
        mml.parameter_scan(parameter)



Overwriting params.py


In [4]:
from where import Where as ML

In [5]:
args.filename

'../data/2019-03-15'

In [6]:
%run params.py

--------------------------------------------------
 parameter scan
--------------------------------------------------
scanning over bias_deconv = [True, False]
For parameter bias_deconv = True ,  

100%|██████████| 10/10 [6:53:59<00:00, 2498.39s/it]


Accuracy=43.2% +/- 0.7%  in 249598.9 seconds
For parameter bias_deconv = False ,  Accuracy=42.7% +/- 0.5%  in 249434.0 seconds
--------------------------------------------------
 base= 2
--------------------------------------------------
--------------------------------------------------
 parameter scan : data
--------------------------------------------------
scanning over sf_0 = [0.1        0.11892071 0.14142136 0.16817928 0.2        0.23784142
 0.28284271 0.33635857 0.4       ]
For parameter sf_0 = 0.100 ,  Accuracy=34.7% +/- 0.6%  in 248961.2 seconds
For parameter sf_0 = 0.119 ,  Accuracy=35.7% +/- 0.5%  in 248985.7 seconds
For parameter sf_0 = 0.141 ,  Accuracy=37.4% +/- 0.5%  in 248932.7 seconds
For parameter sf_0 = 0.168 ,  Accuracy=40.1% +/- 0.7%  in 249173.5 seconds
For parameter sf_0 = 0.200 ,  Accuracy=43.2% +/- 0.7%  in 249200.7 seconds
For parameter sf_0 = 0.238 ,  Accuracy=45.4% +/- 0.7%  in 249060.4 seconds
For parameter sf_0 = 0.283 ,  Accuracy=46.3% +/- 0.7%  in 249112

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

# summary


In [7]:
%ls ../data/_tmp_scanning

B_sf__0_150.npy_lock    B_sf__0_424.npy_lock    sf_0__0_168.npy
B_sf__0_178.npy_lock    bias_deconv__False.npy  sf_0__0_200.npy
B_sf__0_212.npy_lock    bias_deconv__True.npy   sf_0__0_238.npy
B_sf__0_252.npy_lock    sf_0__0_100.npy         sf_0__0_283.npy
B_sf__0_300.npy_lock    sf_0__0_119.npy         sf_0__0_336.npy_lock
B_sf__0_357.npy_lock    sf_0__0_141.npy         sf_0__0_400.npy_lock


# post-processing

In [8]:
!ls  -l {figname}*

UnboundLocalError: local variable 'child' referenced before assignment